In [21]:
# SPDX-FileCopyrightText: 2022 Aleksander Grochowicz
#
# SPDX-License-Identifier: GPL-3.0-or-later

from matplotlib.ticker import AutoMinorLocator
import pandas as pd
from datetime import date
import holidays
from utilities import compute_cdd_hdd_artificial, create_daily_data, create_hourly_data

# Load temperature data

In [22]:
# .txt files from ERA5 notebook
temperatures = pd.read_csv('original_data/temp_moy_Nigeria_2021_2021.txt', header = None)
temperatures.columns = ['FR']
year = 2021
index =  pd.date_range(start='2021-01-01', end='2021-12-31 23:00:00', freq='H')
temperatures.index =index

C:\Users\JD278300\AppData\Local\Temp\ipykernel_9228\819469540.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index =  pd.date_range(start='2021-01-01', end='2021-12-31 23:00:00', freq='H')


# Load regression parameters


In [23]:
reg_parameters_days = pd.read_csv('processing/justine_europe_parameters_days_of_week_2010-2014.csv', index_col = [0])
reg_parameters_hours = pd.read_csv('processing/justine_europe_parameters_hours_of_week_2010-2014.csv', index_col = [0])
reg_parameters_temp = pd.read_csv('processing/justine_europe_parameters_temp_2010-2014.csv', index_col = [0])
reg_parameters_trend = pd.read_csv('processing/justine_europe_parameters_trend_2010-2014.csv', index_col = [0])

FileNotFoundError: [Errno 2] No such file or directory: 'processing/justine_europe_parameters_days_of_week_2010-2014.csv'

In [ ]:
# Dans un premier temps on s'intéresse aux coeffcients de régression pour la France
reg_days_FR = reg_parameters_days['FR']
reg_hours_FR = reg_parameters_hours['FR']
reg_temp_FR = reg_parameters_temp['FR']
reg_trend_FR = reg_parameters_trend['FR']

# reg_days_FR = reg_parameters_days.mean(axis=1)
# reg_hours_FR = reg_parameters_hours.mean(axis=1)
# reg_temp_FR = reg_parameters_temp.mean(axis=1)
# reg_trend_FR = reg_parameters_trend.mean(axis=1)

reg_trend_FR=pd.DataFrame(reg_trend_FR)
reg_days_FR=pd.DataFrame(reg_days_FR)
reg_hours_FR=pd.DataFrame(reg_hours_FR)
reg_temp_FR=pd.DataFrame(reg_temp_FR)

reg_trend_FR.columns = ['FR']
reg_days_FR.columns = ['FR']
reg_hours_FR.columns = ['FR']
reg_temp_FR.columns = ['FR']


In [ ]:
reg_days_FR

,FR
0,45502.300532
1,47091.296189
2,47421.640991
3,47573.598963
4,46838.053120
5,41316.903636
6,38331.676081


In [ ]:
# Dans un premier temps on s'intéresse aux coeffcients de régression pour la France
# reg_days_FR = reg_parameters_days['FI']
# reg_hours_FR = reg_parameters_hours['FI']
# reg_temp_FR = reg_parameters_temp['FI']
# reg_trend_FR = reg_parameters_trend['FI']

# reg_trend_FR=pd.DataFrame(reg_trend_FR)
# reg_days_FR=pd.DataFrame(reg_days_FR)
# reg_hours_FR=pd.DataFrame(reg_hours_FR)
# reg_temp_FR=pd.DataFrame(reg_temp_FR)

# reg_trend_FR.columns = ['KR']
# reg_days_FR.columns = ['KR']
# reg_hours_FR.columns = ['KR']
# reg_temp_FR.columns = ['KR']


Since our validation year is 2015, compared to the training data from 2010 to 2014, all weather years that we use will be processed on the premise that they happened in 2015.

In [ ]:
years = list(temperatures.index.to_series().dt.year.unique())
countries = list(temperatures.columns)

Add holidays as previously with the `holidays` package and by adding Christmas week and for some countries Orthodox Christmas. 

In [24]:
list_holidays = {}
yrs = years.copy()
for c in countries:
    if c == 'EL':
        list_holidays[c] = holidays.CountryHoliday('GR', years = yrs)
    else:
        try:
            list_holidays[c] = holidays.CountryHoliday(c, years = yrs)
        except KeyError:
            list_holidays[c] = holidays.HolidayBase() #if no holidays are available, e.g. ME, MK        

# julian = ['ME', 'MK', 'RS']
# gregorian = countries.copy()
# for i in countries:
#     if i in julian:
#         gregorian.remove(i)

# # Add last wee of the year as holidays.
# for y in list(yrs):
#     for i in gregorian:
#         list_holidays[i].append(date(y,1,2))
#         list_holidays[i].append(list(pd.date_range(start = date(y, 12, 24), end = date(y, 12, 31), freq = '1D')))
#     for i in julian:
#         list_holidays[i].append(list(pd.date_range(start = date(y, 1, 6), end = date(y, 1, 8), freq = '1D')))
        

In [25]:
# #Add German holidays before reunification (so include reunification as everything is assumed to be in 2015)
# overlapping_holidays = ['Neujahrestag', 'Karfreitag', 'Ostermontag', 'Auffahrt', 'Pfingstmontag', 'Weihnachten']
# for y in range(1980, 1991):
#     for date, name in sorted(holidays.CH(years = y).items()):
#         if name in overlapping_holidays:
#             list_holidays['DE'].append({date: name})
#     list_holidays['DE'].append(str(y)+'-05-01')
#     list_holidays['DE'].append(str(y)+'-10-03')
#     list_holidays['DE'].append(str(y)+'-12-26')

In [26]:
# #Add Slovenian holidays before 1992
# AT_SI = ['Neujahr', 'Allerheiligen', 'Stefanitag'] #New Year's, All Saints Day, Independence Day on Boxing Day
# for y in range(1980, 1992):
#     for date, name in sorted(holidays.AT(years = y).items()):
#         if name in AT_SI:
#             list_holidays['SI'].append({date: name})
#     list_holidays['SI'].append(str(y)+'-02-08') #Preseren Day
#     list_holidays['SI'].append(str(y)+'-04-27') #Day of uprising against occupation
#     list_holidays['SI'].append(str(y)+'-05-01') #May 1
#     list_holidays['SI'].append(str(y)+'-05-02') #May 1 over two days
#     list_holidays['SI'].append(str(y)+'-06-25') #Statehood Day

In [27]:
# #Add Bulgarian holidays before 1990
# RO_BG = ['Paștele', 'Anul Nou', 'Ziua Muncii', 'Crăciunul'] #Orthodox Easter, New Year, First of May, Christmas
# for y in range(1980, 1990):
#     for date, name in sorted(holidays.RO(years = y).items()):
#         if name in RO_BG:
#             list_holidays['BG'].append({date: name})
#     list_holidays['BG'].append(str(y)+'-03-03') #Liberation Day
#     list_holidays['BG'].append(str(y)+'-05-06') #Saint George's Day
#     list_holidays['BG'].append(str(y)+'-05-24') #Bulgarian Education and Culture and Slavonic Literature Day
#     list_holidays['BG'].append(str(y)+'-09-06') #Unification Day
#     list_holidays['BG'].append(str(y)+'-09-22') #independence Day

Compute HDD and CDD for all years (with the threshold at 15.5 degrees Celsius).

In [28]:
temperatures_daily = temperatures.resample('1D').mean()
temperatures_daily['weekday'] = temperatures_daily.index.to_series().dt.dayofweek
temperatures_daily['holiday'] = False
temp_daily = {}
for i in countries:
    temp_daily[i] = pd.DataFrame(temperatures_daily[[i, 'weekday', 'holiday']])
    temp_daily[i].columns = ['temp', 'weekday', 'holiday']
temp_with_holidays = temp_daily.copy()
for i in temp_with_holidays.keys():
    for j in temp_with_holidays[i].index:
        if j in list_holidays[i]:
            temp_with_holidays[i].at[j, 'holiday'] = True

In [29]:
daily_hc = compute_cdd_hdd_artificial(temp_daily, countries, threshold_hdd = 15.5, threshold_cdd = 15.5)

In [30]:
daily_hc

{'FR':             heating  weekday  holiday    cooling
 2021-01-01      0.0        4     True  10.955052
 2021-01-02      0.0        5    False  11.407763
 2021-01-03      0.0        6    False  11.709286
 2021-01-04      0.0        0    False  11.996281
 2021-01-05      0.0        1    False  12.315797
 ...             ...      ...      ...        ...
 2021-12-27      0.0        0    False  10.467301
 2021-12-28      0.0        1    False   9.519262
 2021-12-29      0.0        2    False   9.128913
 2021-12-30      0.0        3    False   9.159091
 2021-12-31      0.0        4    False   9.949555
 
 [365 rows x 4 columns]}

Create the artificial demand

In [31]:
countries = list(temperatures.columns)

In [32]:
#Adaptation pour année incomplète comme RTE
import calendar
import numpy as np
def create_daily_data(
    daily_profile,
    trend,
    input_data,
    temp_par,
    place,
    start,
    end,
    validation_days=0,
):
    """Create artificial data based on two regressions from before, one on the weekly load profile and one on the regression on daily values. Also outputs number of days for\
    validation purposes in subsequent period.
    
    Keyword arguments:
    daily_profile -- regression on daily temperature/demand values
    trend -- whether to include trend parameters
    input_data -- temperature data to be used as input
    temp_par -- temperature parameters from regression
    place -- list of keys of data 
    [start, end) -- time period to be studied 
    validation_days -- number of days to move trend for validation runs
    """
    artificial_daily = {}
    first_day = []
    first_day = int(input_data[place[0]].iloc[0]["weekday"])
    days = 0
    for year in range(start, end):
        if calendar.isleap(year) == True:
            days += 366
        else:
            days += 365
    for i in place:
       
        artificial_daily[i] = np.zeros(days)
        for j in range(days):
            k = (j + first_day) % 7
            if input_data[i]["holiday"].iloc[j] == 1:
                artificial_daily[i][j] = (
                    # trend.loc[i] * (validation_days + j)
                    daily_profile[i].iloc[6]
                    + input_data[i].iloc[j]["heating"] * temp_par.loc[i]["par_heating"]
                    + input_data[i].iloc[j]["cooling"] * temp_par.loc[i]["par_cooling"]
                )
            else:
                artificial_daily[i][j] = (
                    # trend.loc[i] * (validation_days + j)
                    daily_profile[i].iloc[k]
                    + input_data[i].iloc[j]["heating"] * temp_par.loc[i]["par_heating"]
                    + input_data[i].iloc[j]["cooling"] * temp_par.loc[i]["par_cooling"]
                )
        artificial_daily[i] = pd.Series(artificial_daily[i], index=input_data[i].index)
    return artificial_daily, days, first_day

In [35]:
reg_trend_FR

,FR
par_trend,-1.376686


In [34]:
# artificial_demand_daily, days, first_day = create_daily_data(reg_parameters_days, reg_parameters_trend.loc["par_trend"], daily_hc, reg_parameters_temp.T, countries, start = 1980, end= 2021, validation_days=1826)
artificial_demand_daily, days, first_day = create_daily_data(reg_days_FR, pd.DataFrame(reg_trend_FR).loc["par_trend"], daily_hc, reg_temp_FR.T, countries, start = 2021, end= 2022)

In [261]:
artificial_demand_hourly = create_hourly_data(artificial_demand_daily, reg_hours_FR, temperatures, first_day, countries, start = 2021, end = 2022)

c:\Users\JD278300\Documents\Stage\Create data\multidecade-data\load data\utilities.py:460: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  0 + artificial[i][j // 24] * weekly_profile[i].iloc[k]


In [262]:
artificial_demand_ts = pd.concat(artificial_demand_hourly, axis = 1).round(1)

In [263]:
artificial_demand_ts

,FR
2021-01-01 00:00:00,37013.6
2021-01-01 01:00:00,36403.3
2021-01-01 02:00:00,34819.7
2021-01-01 03:00:00,33465.2
2021-01-01 04:00:00,33580.6
...,...
2021-12-31 19:00:00,50656.2
2021-12-31 20:00:00,48582.9
2021-12-31 21:00:00,47156.8
2021-12-31 22:00:00,49747.2


In [264]:
artificial_demand_ts.to_csv(f'NIGERIA_demand_artificial_2021-2022.csv')